In [ ]:
import pandas as pd
import numpy as np
def CS_calculate(x,y):
    assert len(x)==len(y),"len(x)!=len(y)"
    zero_lst=[0]*len(x)
    if x == zero_lst or y== zero_lst:
        return float(1) if x==y else float(0)
    res=np.array([[x[i]*y[i],x[i]*x[i],y[i]*y[i]] for i in range(len(x))])
    cos=sum(res[:,0])/(np.sqrt(sum(res[:,1]))*np.sqrt(sum(res[:,2])))
    return cos

def Cosine_Similarity(a,b,df,M):
    region1=[df.iloc[:,0][a]-1,df.iloc[:,1][a]-1]
    region2=[df.iloc[:,0][b]-1,df.iloc[:,1][b]-1]
    vector1=[]
    vector2=[]
    for i in range(len(df)):
        if i!=a and i!=b:
            feature_region=[df.iloc[:,0][i]-1,df.iloc[:,1][i]-1]
            ave_contact1=np.mean(M[region1[0]:region1[-1]+1,feature_region[0]:feature_region[-1]+1])/np.mean(M[region1[0]:region1[-1]+1,region1[0]:region1[-1]+1])
            ave_contact2=np.mean(M[region2[0]:region2[-1]+1,feature_region[0]:feature_region[-1]+1])/np.mean(M[region2[0]:region2[-1]+1,region2[0]:region2[-1]+1])
            vector1.append(ave_contact1)
            vector2.append(ave_contact2)
    CS=CS_calculate(vector1,vector2)
    return CS


In [ ]:
chrs = list(range(20, 23)) 
resolutions = [10000, 25000, 50000, 100000]
for reso in resolutions:
    display_reso = reso // 1000
    for chr in chrs:
        file_path = ("/home/wangxiaoyan/deepTAD/work/all_TADs/bin/deepTAD/VC_normalization/HIC002_%dk_deepTAD.chr%s"%(display_reso,chr))
        with open(file_path, "r") as file:
            lines = file.readlines() 
        nonoverlap_precore = []
        for line in lines:
            column1, column2 = map(int, line.strip().split())
            nonoverlap_precore.append([column1, column2])
        precore = pd.DataFrame(nonoverlap_precore, columns=['left', 'right'])
        nonoverlap_precore = pd.DataFrame(nonoverlap_precore, columns=['left', 'right'])

        matrix = np.loadtxt("/home/wangxiaoyan/deepTAD/prepare_data/VC_normalization/HIC002_%dk_VC.chr%s"%(display_reso,chr))
        datadig0=matrix.copy()
        for i in range(len(matrix)):
            datadig0[i][i]=0
        matrix=datadig0
        average_contact=np.sum(matrix)/len(np.flatnonzero(matrix))

        CS_lst=[]
        CS_threshold=0.8
        for i in range(len(nonoverlap_precore)-1):
            CS = Cosine_Similarity(i,i+1,nonoverlap_precore,matrix)
            CS_lst.append(CS)
        CS_lst.append(0)
        nonoverlap_precore.insert(2,'cosine_similarity',CS_lst)

        row=0
        core_lst=[]

        while(row<len(nonoverlap_precore)):
            if row==len(nonoverlap_precore)-1:
                core_lst.append([nonoverlap_precore['left'][row],nonoverlap_precore['right'][row]])
                row+=1
            else:
                left=nonoverlap_precore['left'][row]
                right=nonoverlap_precore['right'][row]
                CS=nonoverlap_precore['cosine_similarity'][row]
                new_region=[left,right]
                while(CS>=CS_threshold):
                    left2=nonoverlap_precore['left'][row+1]
                    right2=nonoverlap_precore['right'][row+1]
                    if np.mean(matrix[left2-1:right2,nonoverlap_precore['left'][row]-1:nonoverlap_precore['right'][row]])>=average_contact:
                        CS=nonoverlap_precore['cosine_similarity'][row+1]
                        row+=1
                        new_region=[left,right2]
                    else:
                        break
                core_lst.append(new_region)
                row+=1
 
        core_lst = pd.DataFrame(core_lst, columns=['left', 'right'])
        merged_file = pd.concat([precore, core_lst])
        merged_file = merged_file.drop_duplicates().sort_values(by=['left', 'right'])
        # Save the result
        output_path = f"/home/wangxiaoyan/deepTAD/work/deepTAD/VC_normalization/NestedTAD/HIC002_{display_reso}k_deepTAD.chr{chr}"
        merged_file.to_csv(output_path, sep="\t", header=None, index=False)
